In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [3]:
df = pd.read_csv('.\kaggle\LSTM\ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [5]:
print('열의 개수: {}'.format(len(df.columns)))
print(df.columns)

열의 개수: 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any()

False

In [7]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
print('총 샘플의 개수: {}'.format(len(headline)))

총 샘플의 개수: 1324


In [10]:
headline = [n for n in headline if n != 'Unknown']
print('노이즈 값 제거 후 샘플의 개수: {}'.format(len(headline)))

노이즈 값 제거 후 샘플의 개수: 1214


In [11]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [13]:
def repreprocessing(s):
    s=s.encode('utf-8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() #구두점 제거, 소문자
text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [14]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 3494


In [15]:
sequences = list()

for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [18]:
#어떤 정수가 어떤 단어를 의미하는지
index_to_word={} 
for key, value in t.word_index.items(): #인덱스를 단어로 바꾸기 위해 index_to_word 생성
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어 :{}'.format(index_to_word[99]))

빈도수 상위 582번 단어 :former


In [21]:
#패딩작업
max_len=max(len(l) for l in sequences)
print('샘플 최대 길이: {}'.format(max_len))

샘플 최대 길이: 24


In [22]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [23]:
#맨 오른쪽 단어만 분리
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [25]:
#y에 대해 원-핫 인코딩 수행
y = to_categorical(y, num_classes=vocab_size)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 22s - loss: 7.6464 - accuracy: 0.0277
Epoch 2/200
244/244 - 13s - loss: 7.1119 - accuracy: 0.0300
Epoch 3/200
244/244 - 18s - loss: 6.9741 - accuracy: 0.0361
Epoch 4/200
244/244 - 14s - loss: 6.8388 - accuracy: 0.0396
Epoch 5/200
244/244 - 10s - loss: 6.6651 - accuracy: 0.0468
Epoch 6/200
244/244 - 9s - loss: 6.4730 - accuracy: 0.0502
Epoch 7/200
244/244 - 10s - loss: 6.2637 - accuracy: 0.0577
Epoch 8/200
244/244 - 9s - loss: 6.0627 - accuracy: 0.0601
Epoch 9/200
244/244 - 10s - loss: 5.8662 - accuracy: 0.0643
Epoch 10/200
244/244 - 9s - loss: 5.6815 - accuracy: 0.0711
Epoch 11/200
244/244 - 9s - loss: 5.5080 - accuracy: 0.0724
Epoch 12/200
244/244 - 9s - loss: 5.3395 - accuracy: 0.0759
Epoch 13/200
244/244 - 9s - loss: 5.1817 - accuracy: 0.0848
Epoch 14/200
244/244 - 9s - loss: 5.0308 - accuracy: 0.0909
Epoch 15/200
244/244 - 9s - loss: 4.8877 - accuracy: 0.0998
Epoch 16/200
244/244 - 9s - loss: 4.7469 - accuracy: 0.1119
Epoch 17/200
244/244 - 9s - loss: 4.6148 -

Epoch 137/200
244/244 - 9s - loss: 0.3289 - accuracy: 0.9161
Epoch 138/200
244/244 - 9s - loss: 0.3237 - accuracy: 0.9175
Epoch 139/200
244/244 - 9s - loss: 0.3196 - accuracy: 0.9153
Epoch 140/200
244/244 - 9s - loss: 0.3172 - accuracy: 0.9167
Epoch 141/200
244/244 - 9s - loss: 0.3139 - accuracy: 0.9153
Epoch 142/200
244/244 - 9s - loss: 0.3119 - accuracy: 0.9150
Epoch 143/200
244/244 - 9s - loss: 0.3128 - accuracy: 0.9171
Epoch 144/200
244/244 - 9s - loss: 0.3089 - accuracy: 0.9179
Epoch 145/200
244/244 - 9s - loss: 0.3062 - accuracy: 0.9153
Epoch 146/200
244/244 - 9s - loss: 0.3072 - accuracy: 0.9150
Epoch 147/200
244/244 - 9s - loss: 0.3017 - accuracy: 0.9175
Epoch 148/200
244/244 - 9s - loss: 0.3011 - accuracy: 0.9152
Epoch 149/200
244/244 - 9s - loss: 0.2970 - accuracy: 0.9163
Epoch 150/200
244/244 - 10s - loss: 0.2986 - accuracy: 0.9176
Epoch 151/200
244/244 - 9s - loss: 0.3053 - accuracy: 0.9161
Epoch 152/200
244/244 - 9s - loss: 0.3124 - accuracy: 0.9130
Epoch 153/200
244/244 -

In [31]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence=''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    sentence = init_word + sentence
    return sentence

In [33]:
print(sentence_generation(model, t, 'disapprove', 10))

disapprove of squash without the ceiling era as the president peninsula


In [34]:
print(sentence_generation(model, t, 'how', 10))

how to make facebook more accountable about friends he not officials
